In [1]:
from utils import get_openai_client

# OpenAI 클라이언트 생성 (env.txt에서 API 키 자동 로드)
client = get_openai_client()

# API 연결 테스트
completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


### 데이터 전처리 함수 작성


리뷰 JSON 파일 로드

In [2]:
import json

with open('./res/reviews.json', 'r', encoding='utf-8') as f:
    review_list = json.load(f)

review_list[:3]

[{'review': '깨끗하고 너무 좋았어요', 'stars': 5, 'date': '22시간 전'},
 {'review': '위치도 아주좋고 숙소도 깨끗하고 좋아요~', 'stars': 5, 'date': '2일 전'},
 {'review': '잘쉬고놀다갑니다  감사', 'stars': 5, 'date': '6일 전'}]

좋은 평점 및 나쁜 평점 정의
- 좋은 평점 = 별 5개
- 나쁜 평점 = 별 4개 이하

In [3]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(177, 43)

좋은 리뷰 및 나쁜 리뷰로 구분 후 저장

In [4]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_bad[:3]

['[REVIEW_START]인사동, 북촌동, 종각 등등 근처에 갈곳두 많구\r\n숙소두 깨끗하구 너무 좋아요\r\n엘베를 2번 타야하는게 살짝 번거롭지만\r\n그거빼구 다 좋아요[REVIEW_END]',
 '[REVIEW_START]근처 볼거리 놀거리가 가까이 있어 편했으나 \n조계사 고층뷰는 의미 없이 배정해 주었으며\n객실 내 청결상태가 별로였음 욕조 머리카락, 수건의 먼지등\n재방문 의사 없음[REVIEW_END]',
 '[REVIEW_START]2월21일(수) 오전7시경, 카운터에 계시던 남자 직원 불친절\n방음 최악[REVIEW_END]']

In [5]:
reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)

reviews_bad_text[:100]

'[REVIEW_START]인사동, 북촌동, 종각 등등 근처에 갈곳두 많구\r\n숙소두 깨끗하구 너무 좋아요\r\n엘베를 2번 타야하는게 살짝 번거롭지만\r\n그거빼구 다 좋아요[REVIEW_'

전처리 함수 작성

In [6]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

good, bad = preprocess_reviews()
good[:100]

'[REVIEW_START]깨끗하고 너무 좋았어요[REVIEW_END]\n[REVIEW_START]위치도 아주좋고 숙소도 깨끗하고 좋아요~[REVIEW_END]\n[REVIEW_STAR'

### 평가용 함수 작성

평가 기준 설정
- MT-Bench 논문 기반 Pairwise Comparision (=LLM 기반 평가)
  - 비교하는 방식 vs. 점수 매기는 방식
  - 점수라는게 애매 할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)
  - 경험상 점수보다는 비교가 상대적으로 더 정확한 편

평가 스크립트 작성
- MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summaries, accommodations)

In [7]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
    
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion

### Baseline 모델 개발

In [8]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [9]:
# PROMPT_BASELINE 정의
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

reviews, _ = preprocess_reviews(path='./res/reviews.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

숙소는 깨끗하고 위치도 좋았으며 편안한 휴식을 즐겼다고 감사의 말을 전했다.


In [10]:
summary_real_20240526 = '위치가 매우 우수한 숙박시설로, 인사동과 조계사, 경복궁 등 관광지에 도보로 이동할 수 있는 편리한 위치에 있습니다. 객실은 깔끔하며 직원들의 친절한 서비스와 청결한 시설이 인상적입니다. 주변에는 맛집과 편의시설이 많아 편리하며, 교통 접근성도 좋습니다. 전체적으로 만족도가 높고 자주 방문하고 싶은 곳으로 손꼽히는 숙소로 평가됩니다.'

In [11]:
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE).choices[0].message.content, summary_real_20240526).choices[0].message.content)

Assistant A provides a concise summary that accurately reflects the user reviews. It mentions the cleanliness, good location, and the fact that the guests enjoyed their stay and expressed gratitude. This summary is directly relevant to the reviews provided.

Assistant B, on the other hand, includes a lot of additional information that is not present in the user reviews. While the response is detailed and creative, it introduces elements such as specific tourist attractions, staff service, and nearby amenities, which are not mentioned in the original reviews. This makes the summary less accurate and less relevant to the given reviews.

Therefore, Assistant A better follows the user’s instructions by providing a summary that is helpful, relevant, accurate, and directly based on the user reviews.

[[A]]


### 대규모 평가 스크립트
- 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt에 대해 temperature 높여서 평가 진행

In [12]:
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summaries_baseline

['숙소는 깨끗하고 위치가 좋았으며 잘 쉬고 놀다감을 표현했다.',
 '이 숙소는 깨끗하고 위치도 좋아서 잘 쉬고 놀 수 있었다고 감사한 리뷰를 남겼다.',
 '숙소는 깨끗하고 위치도 좋았으며 편안히 쉬고 즐거운 시간을 보냈다고 만족하는 리뷰다.',
 '숙소는 깨끗하고 위치도 좋았으며 편안한 휴식을 즐길 수 있었던 만족스러운 숙박이었습니다.',
 '이 숙소는 깨끗하고 좋은 위치에 있어서 편안한 휴식을 즐길 수 있었습니다. 감사합니다.',
 '숙소는 매우 깨끗하고 위치도 아주 좋았으며 편안한 휴식을 취할 수 있었다고 감사의 말을 전했다.',
 '숙소는 깨끗하고 위치도 좋았으며 편안한 휴식을 취할 수 있었던 것 같다.',
 '숙소는 깨끗하고 위치도 좋았으며 잘 쉬었던 만족스러운 숙박이었습니다.',
 '숙소는 깨끗하고 위치도 좋았으며 잘 쉬고 놀 수 있었다.',
 '깨끗하고 위치도 좋았어요. 잘 쉬고 놀다 간다고 합니다. 감사합니다.']

In [13]:
from tqdm import tqdm


def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = completion.choices[0].message.content

        if '[[A]]' in verdict_text:
            a_cnt += 1
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        else:
            print('Evaluation Error')

    return a_cnt, b_cnt, draw_cnt

wins, losses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_real_20240526 for _ in range(len(summaries_baseline))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:18<00:00,  1.83s/it]

Wins: 10, Losses: 0, Ties: 0


### 모델 고도화 1 - 조건들 명시

In [14]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:18<00:00,  1.87s/it]

Wins: 10, Losses: 0, Ties: 0


### 모델 고도화 2 - 입력 데이터의 품질 증가

In [15]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    filtered_cnt = 0
    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue
        if len(r['review']) < 30:
            filtered_cnt += 1
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    reviews_good = reviews_good[:min(len(reviews_good), 50)]
    reviews_bad = reviews_bad[:min(len(reviews_bad), 50)]

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

reviews, _ = preprocess_reviews()

In [16]:
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:20<00:00,  2.10s/it]

Wins: 3, Losses: 7, Ties: 0


### 모델 고도화 3 - Few-Shot Prompting

In [17]:
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:25<00:00,  2.50s/it]

Wins: 2, Losses: 8, Ties: 0


In [18]:
summaries

['이 숙소는 전체적으로 깨끗하고 편안한 분위기를 제공합니다. 친절한 직원들이 좋은 서비스를 제공해주어 머무는 동안 편안했습니다. 객실은 넓고 아늑하며 휴식을 취하기에 좋은 환경을 제공합니다. 다음 여행 때도 재방문하고 싶은 좋은 숙소입니다. 감사합니다.',
 '리뷰: 이 호텔은 넓은 객실과 편안한 침대로 기분 좋게 머물렀습니다. 직원들은 친절하고 응대가 좋았어요.\n\n요약: 넓은 객실과 편안한 침대로 기분 좋게 머물렀고, 직원들의 친절한 서비스가 인상적이었습니다. 또한, 숙소의 편안한 분위기가 좋은 인상을 주었습니다.',
 '숙소는 전반적으로 깔끔하고 편안한 분위기를 가졌습니다. 친절한 스태프들이 좋은 서비스를 제공해주었고, 위치 또한 편리하게 위치해 있었습니다. 객실은 청결하고 편리한 시설들로 구비되어 있어 휴식을 취하기에 안성맞춤이었습니다. 다음 방문에도 이 숙소를 선택할 예정입니다. 감사합니다.',
 '1. 직원들이 친절하고 응대도 좋았습니다.\n2. 객실은 깨끗하고 편안한 분위기를 자랑합니다.\n3. 편리한 위치에 위치해 있어 여행을 즐기는 데 도움이 되었습니다.\n4. 다음에 또 이 숙소를 이용하고 싶다는 마음이 들었습니다.',
 '이 숙소는 전반적으로 깔끔하고 아늑한 분위기를 자랑합니다. 객실은 청결하며 편안한 침대가 마음에 들었습니다. 또한 위치가 편리하여 다시 방문하고 싶은 숙소입니다. 다음에 또 머물 의향이 있는 리뷰들이 있습니다. 고객들은 대부분 만족한 경향이 있어 추천할 만한 숙소라고 생각됩니다.',
 '이 숙소는 전반적으로 깨끗하고 청결한 분위기가 느껴졌습니다. 직원들의 친절한 서비스도 좋았고, 편안한 숙면을 취할 수 있었습니다. 또한, 주변에 편의시설이 많아 편리했고, 재방문하고 싶은 숙소였습니다. 강력 추천합니다.',
 '리뷰 1: 아늑한 분위기와 청결한 시설에 만족했습니다.\n\n리뷰 2: 위치도 좋고 직원들의 서비스가 좋았어요.\n\n리뷰 3: 조식이 맛있었고 편안한 휴식을 취할 수 있었습니다.\n\n리뷰 4: 다음에 또 머물고

In [19]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_1shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:26<00:00,  2.69s/it]

Wins: 10, Losses: 0, Ties: 0


In [20]:
summaries

['[REVIEW_START]이번에 처음 방문했는데, 직원들의 친절함에 감동받았습니다. 숙소 내부도 깔끔하고 편안했어요. 조식도 맛있었습니다. 다음에 또 방문하고 싶네요![REVIEW_END]\n[REVIEW_START]위치가 시내 중심에 있어서 이동이 편리하고 주변에 맛집들이 많아서 좋았습니다. 방은 조용해서 편안했고, 침구도 푹신해서 좋았어요. 다음에 또 방문할 의향 있습니다![REVIEW_END] \n\n요약 결과:\n이 숙소는 직원들의 친절함과 깔끔하고 편안한 내부, 맛있는 조식으로 손님들을 매료시키고 있습니다. 또한, 시내 중심에 위치해 이동이 편리하고 주변 맛집들이 많아서 만족도가 높은 편입니다. 숙소는 조용하고 침구도 편안해서 재방문을 고려하고 있는 손님이 많습니다.',
 '1. [REVIEW_START]객실이 깔끔하고 편안해서 휴식을 취하기에 좋았습니다. 조식도 맛있었어요![REVIEW_END]\n2. [REVIEW_START]직원들이 친절하고 서비스가 좋았습니다. 다음에 또 방문하고 싶은 곳이에요.[REVIEW_END]\n3. [REVIEW_START]위치가 시내 중심가와 가깝고 주변에 편의시설이 많아서 편리했습니다. 전체적으로 만족합니다![REVIEW_END] \n\n요약 결과:\n이 숙소는 깔끔하고 편안한 객실과 맛있는 조식을 제공하며, 친절한 직원들과 좋은 서비스로 손님들을 맞이하고 있습니다. 또한, 시내 중심가와 가까운 위치에 있어 편리하고 주변에 다양한 편의시설이 있어 편안한 휴식을 취할 수 있다는 평가를 받고 있습니다.',
 '1. [REVIEW_START]이번에 처음 방문했는데 직원분들이 친절하고 시설도 깔끔해서 좋았어요. 다음에 또 방문하고 싶어요![REVIEW_END]\n2. [REVIEW_START]가족들과 함께 머물렀는데, 아이들이 수영장을 너무 좋아했어요. 주변에 놀 거리도 많아서 즐거운 시간을 보냈습니다![REVIEW_END]\n\n요약 결과:\n이 숙소는 처음 방문한 고객들에게 직원의 친절과 깔끔한 시설로 호평을 받고 

In [21]:
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

In [22]:
reviews_2shot, _ = preprocess_reviews('./res/ninetree_yongsan.json')
summary_2shot = summarize(reviews_2shot, prompt_1shot, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.

예시 리뷰들 1:
{reviews_1shot}
예시 요약 결과 1:
{summary_1shot}

예시 리뷰들 2:
{reviews_2shot}
예시 요약 결과 2:
{summary_2shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_2shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:20<00:00,  2.04s/it]

Wins: 0, Losses: 10, Ties: 0


In [23]:
summaries

['죄송하지만 숙소 리뷰가 제공되지 않았습니다. 숙소 리뷰를 제공해주시면 요약을 도와드리겠습니다.',
 '죄송하지만, 숙소 리뷰들을 제공해주지 않으셨습니다. 숙소 리뷰들을 입력해주시면 요약을 도와드리겠습니다.',
 '죄송하지만, 숙소 리뷰가 제공되지 않아 요약할 수 없습니다. 숙소 리뷰를 제공해주시면 감사하겠습니다.',
 '죄송하지만 숙소 리뷰가 제공되지 않았습니다. 숙소 리뷰를 입력해주시면 요약을 도와드리겠습니다.',
 '죄송하지만, 숙소 리뷰가 제공되지 않아요. 숙소 리뷰를 제공해주시면 요약을 도와드리겠습니다.',
 '죄송하지만 숙소 리뷰가 제공되지 않아 요약을 도와드릴 수 없습니다. 숙소 리뷰를 제공해주시면 감사하겠습니다.',
 '죄송하지만 숙소 리뷰가 제공되지 않아 요약을 도와드릴 수 없습니다. 숙소 리뷰를 제공해주시면 감사하겠습니다.',
 '죄송하지만 숙소 리뷰들을 제공해주시지 않아 요약을 제공해드릴 수 없습니다. 숙소 리뷰들을 입력해주시면 해당 리뷰들을 요약해드리겠습니다.',
 '죄송하지만, 숙소 리뷰가 제공되지 않았습니다. 숙소 리뷰를 입력해 주시면 요약을 도와드리겠습니다.',
 '죄송하지만, 숙소 리뷰가 제공되지 않았습니다. 숙소 리뷰를 제공해주시면 요약을 도와드리겠습니다.']